# Accuracy Assessment
This notebook is intended to determine which interpolation method performs better. Based on the results, we can see that IDW has the best performance. The data used for testing was precipitation.

### Interpolation algorithms results

In [19]:
# IDW
prep_idw = "Idw"

# ORDINARY_KRIGING
prep_ord_kri = "kriging_ord"

# UNIVERSAL_KRIGING
prep_uni_kri = "kriging_uni"

In [24]:
def differences(true_points, interpolated_raster, interpolated_points):

    # convert raster to point
    arcpy.conversion.RasterToPoint(
        in_raster=interpolated_raster,
        out_point_features=os.path.join(arcpy.env.workspace, interpolated_points),
        raster_field="Value"
    )   
    
    # get the name of interpolation method as part of the output point layer 
    interpolation = os.path.basename(interpolated_points)
    
    # spatal join
    output_join = f"diff_{interpolation}"
    arcpy.analysis.SpatialJoin(true_points, interpolated_points, output_join, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "CLOSEST")

    # rename column
    arcpy.management.AlterField(output_join, "Valor", "ground_truth", "ground_truth")
    arcpy.management.AlterField(output_join, "CodigoEstacion", "station", "station")
    arcpy.management.AlterField(output_join, "grid_code", "prediction", "prediction")

    # calculate difference
    fields = ["station", "ground_truth", "prediction", "residual"]
    arcpy.management.AddField(output_join, "residual", "DOUBLE")
    with arcpy.da.UpdateCursor(output_join, fields) as cursor:
        for row in cursor:
            row[3] = row[1] - row[2]  # "ground_truth" - "prediction"
            cursor.updateRow(row)
    arcpy.management.DeleteField(output_join,"Join_Count;Longitud;Latitud;TARGET_FID;pointid","DELETE_FIELDS")
    
    print(f"Difference {interpolation} points layer created.")


In [26]:
true_points = "precipitation_monthly_2013_23_Clip"

# ORDINARY_KRIGING
differences(true_points, prep_ord_kri, prep_ord_kri+"_pts")
# UNIVERSAL_KRIGING
differences(true_points, prep_uni_kri, prep_uni_kri+"_pts")
# IDW
differences(true_points, prep_idw, prep_idw+"_pts")

Difference kriging_ord_pts points layer created.
Difference kriging_uni_pts points layer created.
Difference Idw_pts points layer created.


## Defince the RMSE, MAE, R^2 function

In [27]:
# RMSE
def RMSE(data):
    residual_squared_sum = 0
    count = 0
    for row in arcpy.da.SearchCursor(data, ['residual']):
        residual_squared_sum += row[0] ** 2
        count += 1
            
    rmse = (residual_squared_sum / count) ** 0.5
    return rmse


In [28]:
# Mean Absolute Error
def MAE(data):
    absolute_errors = []
    with arcpy.da.SearchCursor(data, ["ground_truth", "prediction"]) as cursor:
        for row in cursor:
            ground_truth = row[0]
            prediction = row[1]
            absolute_errors.append(abs(ground_truth - prediction))
    
    
    mae = sum(absolute_errors) / len(absolute_errors)
    return mae

In [29]:
# Coefficient Of Determination
def R_squared(data):
    y_true = []
    y_pred = []
    
    with arcpy.da.SearchCursor(data, ["ground_truth", "prediction"]) as cursor:
        for row in cursor:
            y_true.append(row[0])
            y_pred.append(row[1])
    
    y_mean = sum(y_true) / len(y_true)    
    ss_total = sum((y_i - y_mean) ** 2 for y_i in y_true)   
    ss_residual = sum((y_true[i] - y_pred[i]) ** 2 for i in range(len(y_true)))   
    r_squared = 1 - (ss_residual / ss_total)   
    return r_squared

     


In [30]:
# execute the function
diff_agdd_interpolation = ["diff_Idw_pts", "diff_kriging_uni_pts", "diff_kriging_ord_pts"]

for diff_agdd in diff_agdd_interpolation:
    
    print(f"{diff_agdd} RMSE:", RMSE(diff_agdd))
    
    print(f"{diff_agdd} MAE:", MAE(diff_agdd))

    print(f"{diff_agdd} R^2:", R_squared(diff_agdd))
     

diff_Idw_pts RMSE: 0.0024740672886870577
diff_Idw_pts MAE: 0.001817189069839742
diff_Idw_pts R^2: 0.9999999999424458
diff_kriging_uni_pts RMSE: 459.1004665602453
diff_kriging_uni_pts MAE: 283.78659801614396
diff_kriging_uni_pts R^2: -0.9818442010040875
diff_kriging_ord_pts RMSE: 0.48543483155341316
diff_kriging_ord_pts MAE: 0.4148387168736747
diff_kriging_ord_pts R^2: 0.999997784274722
